In [1]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.6.0
2.6.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

NGRAMS = 2
SAMPLE = 1000000
EPOCHS = 15

# Florida voter
df = pd.read_csv('../dataverse_files/fl_reg_name_race.csv.gz')
df.dropna(subset=['name_first', 'name_last'], inplace=True)
sdf = df[df.race.isin(['multi_racial', 'native_indian', 'other', 'unknown']) == False].sample(SAMPLE, random_state=21)
del df

# Additional features
sdf['name_first'] = sdf.name_first.str.title()
sdf['name_last'] = sdf.name_last.str.title()

sdf

,name_last,name_first,race
841323,Torres,Jose,hispanc
1408926,Da Silva,Amanda,nh_white
1733118,Mc Ghee,Sandra,nh_white
13104513,Karam,Melinda,nh_white
9156114,Brewer,Laia,nh_black
...,...,...,...
3076722,Antunez Avila,Robert,hispanc
10023679,Davis,Wyatt,nh_white
5846252,Scott,Jacquelyn,nh_white
5959131,Parton,Douglas,nh_white


In [3]:
rdf = sdf.groupby('race').agg({'name_first': 'count'})
rdf.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_name_race.csv', columns=[])
rdf

,name_first
race,
asian,19431
hispanc,166865
nh_black,142675
nh_white,671029


In [4]:
sdf.groupby('race').agg({'name_last': 'nunique'})

,name_last
race,
asian,9364
hispanc,41721
nh_black,23079
nh_white,145852


## Preprocessing the input data

In [5]:
# concat last name and first name
sdf['name_last_name_first'] = sdf['name_last'] + ' ' + sdf['name_first']

# build n-gram list
vect = CountVectorizer(analyzer='char', max_df=0.3, min_df=3, ngram_range=(NGRAMS, NGRAMS), lowercase=False) 
a = vect.fit_transform(sdf.name_last_name_first)
vocab = vect.vocabulary_

# sort n-gram by freq (highest -> lowest)
words = []
for b in vocab:
    c = vocab[b]
    #print(b, c, a[:, c].sum())
    words.append((a[:, c].sum(), b))
    #break
words = sorted(words, reverse=True)
words_list = [w[1] for w in words]
num_words = len(words_list)
print("num_words = %d" % num_words)


def find_ngrams(text, n):
    a = zip(*[text[i:] for i in range(n)])
    wi = []
    for i in a:
        w = ''.join(i)
        try:
            idx = words_list.index(w)
        except:
            idx = 0
        wi.append(idx)
    return wi

# build X from index of n-gram sequence
X = np.array(sdf.name_last_name_first.apply(lambda c: find_ngrams(c, NGRAMS)))

# check max/avg feature
X_len = []
for x in X:
    X_len.append(len(x))

max_feature_len = max(X_len)
avg_feature_len = int(np.mean(X_len))

print("Max feature len = %d, Avg. feature len = %d" % (max_feature_len, avg_feature_len))
y = np.array(sdf.race.astype('category').cat.codes)

# Split train and test dataset
X_train,  X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

num_words = 1268
Max feature len = 36, Avg. feature len = 12


## Train a LSTM model

ref: http://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/

In [6]:
'''The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:

- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.

- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Activation
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.models import load_model

max_features = num_words # 20000
feature_len = 25 # avg_feature_len # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=feature_len)
X_test = sequence.pad_sequences(X_test, maxlen=feature_len)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

num_classes = np.max(y_train) + 1
print(num_classes, 'classes')

print('Convert class vector to binary class matrix '
      '(for use with categorical_crossentropy)')
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

800000 train sequences
200000 test sequences
Pad sequences (samples x time)
X_train shape: (800000, 25)
X_test shape: (200000, 25)
4 classes
Convert class vector to binary class matrix (for use with categorical_crossentropy)
y_train shape: (800000, 4)
y_test shape: (200000, 4)


In [7]:
print('Build uncertainty model...')

#model_uncrtn = Sequential()
input_ = keras.layers.Input(shape=(feature_len))
layer_ = Embedding(num_words, 32) (input_)
layer_ = LSTM(128, dropout=0.2, recurrent_dropout=0.2)(layer_, training=True)
output = Dense(num_classes, activation='softmax') (layer_)

model_uncrtn = keras.models.Model(inputs=input_, outputs=output)

# try using different optimizers and different optimizer configs
model_uncrtn.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model_uncrtn.summary())

Build uncertainty model...
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 25)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 25, 32)            40576     
_________________________________________________________________
lstm (LSTM)                  (None, 128)               82432     
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 123,524
Trainable params: 123,524
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
y_train[:10]

array([[0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 0., 0., 1.],
       [0., 1., 0., 0.]], dtype=float32)

In [9]:
print('Train...')
model_uncrtn.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS,
          validation_split=0.1, verbose=1)
score, acc = model_uncrtn.evaluate(X_test, y_test,
                            batch_size=batch_size, verbose=1)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/15
22500/22500 [==============================] - 636s 28ms/step - loss: 0.5663 - accuracy: 0.7946 - val_loss: 0.5297 - val_accuracy: 0.8074
Epoch 2/15
22500/22500 [==============================] - 571s 25ms/step - loss: 0.5117 - accuracy: 0.8159 - val_loss: 0.5059 - val_accuracy: 0.8193
Epoch 3/15
22500/22500 [==============================] - 583s 26ms/step - loss: 0.4915 - accuracy: 0.8239 - val_loss: 0.4919 - val_accuracy: 0.8254
Epoch 4/15
22500/22500 [==============================] - 572s 25ms/step - loss: 0.4803 - accuracy: 0.8283 - val_loss: 0.4866 - val_accuracy: 0.8287
Epoch 5/15
22500/22500 [==============================] - 570s 25ms/step - loss: 0.4740 - accuracy: 0.8304 - val_loss: 0.4839 - val_accuracy: 0.8286
Epoch 6/15
22500/22500 [==============================] - 581s 26ms/step - loss: 0.4697 - accuracy: 0.8323 - val_loss: 0.4818 - val_accuracy: 0.8298
Epoch 7/15
22500/22500 [==============================] - 567s 25ms/step - loss: 0.4657 - accurac

In [10]:
print('Test score:', score)
print('Test accuracy:', acc)

Test score: 0.4723474979400635
Test accuracy: 0.832360029220581


## Confusion Matrix

In [12]:
p = model_uncrtn.predict(X_test, verbose=2) # to predict probability
y_pred = np.argmax(p, axis=-1)
target_names = list(sdf.race.astype('category').cat.categories)
print(classification_report(np.argmax(y_test, axis=1), y_pred, target_names=target_names))
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

6250/6250 - 30s
              precision    recall  f1-score   support

       asian       0.72      0.46      0.56      3886
     hispanc       0.83      0.82      0.82     33373
    nh_black       0.73      0.42      0.53     28535
    nh_white       0.85      0.94      0.89    134206

    accuracy                           0.83    200000
   macro avg       0.78      0.66      0.70    200000
weighted avg       0.82      0.83      0.82    200000

[[  1771    457    187   1471]
 [   127  27336    491   5419]
 [   153    592  11850  15940]
 [   405   4541   3694 125566]]


## Save model

In [16]:
model_uncrtn.save('../dataverse_files/fl_voter_reg/lstm/fl_all_name_lstm_uncrtn.h5')

In [17]:
words_df = pd.DataFrame(words_list, columns=['vocab'])
words_df.to_csv('../dataverse_files/fl_voter_reg/lstm/fl_all_name_vocab.csv', index=False, encoding='utf-8')
